In [7]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
import pandas as pd
from collections import Counter

def dist2D(pos1, pos2):
    return np.sqrt((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)

In [2]:
filename = '20221104_0051.npy'
traj = np.load(f'{filename}', allow_pickle=True).item()
lenFrames = traj['trajectories'].shape[0]
Nfish = traj['trajectories'].shape[1]

# traj of the center of mass:
trajCM = np.sum(traj['trajectories'], axis=1)/Nfish

traj['setup_points']
print(-traj['setup_points']['corners_out'][0,0]+traj['setup_points']['corners_out'][1,0])
print(-traj['setup_points']['corners_out'][3,0]+traj['setup_points']['corners_out'][2,0])

2755
2876


In [13]:
trajCM[1]

array([2773.83190063, 1000.77842147])

In [14]:
trajCM[1,0]

2773.83190062752

In [8]:
# min_dists = []
# max_dists = []
max_dists_to_CM = []
max_time = 50000
# max_time = lenFrames
for t in range(max_time):
    dists = []
    dists_to_CM = []
    for f in range(Nfish-1):
        dists_to_CM.append(dist2D(traj['trajectories'][t,f,:], trajCM[t,:]))
        # for g in range(f+1,Nfish):
        #     d = dist2D(traj['trajectories'][t,f,:], traj['trajectories'][t,g,:])
        #     dists.append(d)
    # min_dists.append(min(dists))
    # max_dists.append(max(dists))
    dists_to_CM.append(dist2D(traj['trajectories'][t,Nfish-1,:], trajCM[t,:]))
    max_dists_to_CM.append(max(dists_to_CM))

In [9]:
print(f'Average max dist to the CM: {np.average(max_dists_to_CM)}')

Average max dist to the CM: 321.69634066090845


### I should also check how much a fish moves from one frame to another

In [ ]:
# ...

In [10]:
# build box considering the average distance to the CM
avgMaxDistCM = np.average(max_dists_to_CM)
boxSide = 10
Lint = ceil(avgMaxDistCM*2/boxSide)
print(f'Number of boxes per side: {Lint}')
L = Lint*boxSide
print(f'Box side: {L}')
print(f'which should approx correspond to: {avgMaxDistCM*2}')

Nboxs = Lint**2
print(f'Total number of boxes: {Nboxs}')

Number of boxes per side: 65
Box side: 650
which should approx correspond to: 643.3926813218169
Total number of boxes: 4225


# box occupancies

In [22]:
frames, colrows = [], []
# for t in range(max_time):
trialTime = 1000
for t in range(trialTime):
    colrows.append({})
    for f in traj['trajectories'][t]:
        col = int((f[0]-trajCM[t,0]+L/2)/boxSide)
        row = int((f[1]-trajCM[t,1]+L/2)/boxSide)
        if (col, row) not in colrows[t].keys():
            colrows[t][(col, row)] = 1
        else:
            colrows[t][(col, row)] += 1

### build an occupancy chain

In [39]:
colrows_all_frames = []
for t in range(trialTime):
    colrows_all_frames.extend(list(colrows[t].keys()))
count = Counter(colrows_all_frames)

In [40]:
count

Counter({(30, 22): 52,
         (22, 20): 42,
         (30, 33): 40,
         (39, 23): 39,
         (32, 34): 36,
         (30, 37): 36,
         (37, 47): 36,
         (47, 61): 36,
         (36, 33): 34,
         (30, 32): 32,
         (33, 34): 31,
         (38, 19): 31,
         (39, 24): 30,
         (32, 35): 30,
         (30, 50): 30,
         (40, 37): 29,
         (20, 33): 28,
         (34, 35): 28,
         (23, 43): 28,
         (41, 36): 28,
         (40, 24): 28,
         (27, 42): 27,
         (46, 5): 27,
         (33, 35): 27,
         (36, 23): 27,
         (26, 42): 26,
         (31, 35): 26,
         (29, 34): 26,
         (34, 31): 25,
         (34, 39): 25,
         (30, 36): 24,
         (30, 51): 24,
         (36, 47): 24,
         (35, 27): 23,
         (32, 14): 23,
         (34, 12): 23,
         (13, 30): 23,
         (30, 46): 23,
         (28, 34): 23,
         (19, 34): 22,
         (32, 28): 22,
         (18, 34): 22,
         (36, 24): 22,
         (34

In [41]:
# e.g. for box
col, row = 30, 22
# box_occ = np.zeros(trialTime)
box_occ = []
for t in range(trialTime):
    box_occ.append(1) if (col,row) in colrows[t].keys() else box_occ.append(0)


In [43]:
sum(box_occ)

52